In [7]:
from http.client import responses

import requests

params = {
  "city": "Paris",
  "countryCode": "FR",
  "locale": "*",
  "classificationName": "Sports"
}

url = "https://app.ticketmaster.com/discovery/v2/events.json"
params["apikey"] = "97ViRGFgMAAjpbWjitt2p3JlLNfoMYQz"

all_events = []

response = requests.get(url, params=params)
    
response.json()

{'_embedded': {'events': [{'name': 'PARIS BASKETBALL VS ZALGIRIS KAUNAS',
    'type': 'event',
    'id': 'rZ7SnyZ1Ad0fAg',
    'test': False,
    'description': "PARIS BASKETBALL VS ZALGIRIS KAUNAS ADIDAS ARENA - PARIS Venez vivre pleinement l'expérience Paris Basketball pour cette 9ème journée de Coupe d'Europe face au Zalgiris Kaunas à partir de 16€ !",
    'url': 'https://www.ticketmaster.fr/fr/manifestation/paris-basketball-vs-zalgiris-kaunas-billet/idmanif/602975/idtier/18864121',
    'locale': 'fr-fr',
    'images': [{'ratio': '16_9',
      'url': 'https://s1.ticketm.net/dam/c/8d5/f95bdd17-1d94-4e98-9295-641e4db558d5_105621_RECOMENDATION_16_9.jpg',
      'width': 100,
      'height': 56,
      'fallback': True},
     {'ratio': '3_2',
      'url': 'https://s1.ticketm.net/dam/c/8d5/f95bdd17-1d94-4e98-9295-641e4db558d5_105621_ARTIST_PAGE_3_2.jpg',
      'width': 305,
      'height': 203,
      'fallback': True},
     {'ratio': '16_9',
      'url': 'https://s1.ticketm.net/dam/c/8d5/f

In [60]:
import requests

def request_ticket_master(params) -> str:
    """
    Found all events from TicketMaster API
    :param params:
    :return:
    """
    url = "https://app.ticketmaster.com/discovery/v2/events.json"
    params["apikey"] = "97ViRGFgMAAjpbWjitt2p3JlLNfoMYQz"

    all_events = []
    while True:
        response = requests.get(url, params=params)
        if response.status_code != 200:
            print("Erreur lors de la récupération des données :", response.status_code)
            break

        data = response.json()
        events = data.get("_embedded", {}).get("events", [])

        if not events:
            print("Toutes les pages ont été récupérées.")
            break

        for event in events:
            event_info = {
                "name": event['name'],
                "date": event['dates']['start']['localDate'],
                "venue": event['_embedded']['venues'][0]['name'],
                "description": event.get('description', "Aucune description disponible"),
                "url": event.get("url", "Lien non disponible")  # Ajout du lien de l'événement
            }

            price_ranges = event.get("priceRanges")
            if price_ranges:
                min_price = price_ranges[0].get("min")
                max_price = price_ranges[0].get("max")
                currency = price_ranges[0].get("currency", "EUR")
                event_info["price"] = f"de {min_price} à {max_price} {currency}"
            else:
                event_info["price"] = "Informations non disponibles"

            all_events.append(event_info)

        params["page"] += 1

    with open("output.txt", "w", encoding="utf-8") as file:
        for event in all_events:
            file.write(f"Nom de l'événement: {event['name']}\n")
            file.write(f"Date: {event['date']}\n")
            file.write(f"Lieu: {event['venue']}\n")
            file.write(f"Description: {event['description']}\n")
            file.write(f"Prix: {event['price']}\n")
            file.write(f"Lien Ticketmaster: {event['url']}\n")  # Ajout du lien dans le fichier
            file.write("-" * 20 + "\n")

    print("Les informations des événements ont été enregistrées dans 'output.txt'.")


Erreur lors de la récupération des données : 400
Nom de l'événement: WHAT'S LOVE GOT TO DO WITH IT ?
Date: 2024-11-10
Lieu: SALLE PLEYEL
Description: Après une première tournée à guichets fermés à travers le Royaume-Uni, en Europe et aux Etats-Unis, le tribute de Tina Turner et son show « What's Love Got To Do With It ? » traverse le Channel pour la première fois pour célébrer cette incroyable légende du rock et de la soul.Le show « What's Love Got To Do With It ? », lancé en février 2019, rend un hommage remarquable et remarqué à Tina Turner, qui fait partie avec Diana Ross et Aretha Franklin des chanteuses afro-américaines les plus influentes du xxe siècle. Première artiste noire et première femme à faire la couverture du magazine Rolling Stone, elle fut deux fois intronisée au Rock and Roll Hall of Fame  et remporta 12 Grammy Awards. Le concert retrace les 50 ans de carrière de l'une des artistes les plus populaires au monde pendant lequel sont interprétés en live  tous les succès d

### user-agent

In [79]:
import ollama

In [82]:
with open("output.txt", "r", encoding="utf-8") as file:
    content = file.read()

modelfile = (
    "FROM llama3.1\n"
    "SYSTEM Tu es un agent de voyage qui génère une liste d'evenement qui va aider à planifier un séjour. "
    "À partir d'une liste d'événements, tu dois donner le top 3 qui correspondent à la demande de l'utilisateur."
)

ollama.create(model='agent2', modelfile=modelfile)


{'status': 'success'}

In [83]:
stream = ollama.chat(
    model='agent2',
    messages=[
        {
            'role': 'user',
            'content': (
                f"Voici les informations récupérées : {content}."
                "voici la query de l'utilisateur : 'je veux partir à Paris et je veux assister à un concert de musique un peu electrique."
                "Tu dois donner le top 3 des événements qui correspondent à la demande de l'utilisateur et Propose moi que des dates différentes pour chaque événement. "
                "et qui sont dans l'intervalle de dates demandé.\n\n"
                "Voici un format du document que j’attends par exemple :\n\n"
                "1. Event: DUO BOUCHER – CHARLESTON SUR LA TOUR Eiffel\n"
                "   Date: 2022-01-05\n"
                "   Lieu: Le Trianon\n"
                "   Description: Duo Boucher, composé de Louis et Thomas Boucher, présente son nouvel album "
                "   Lien Ticketmaster : https://www.ticketmaster.fr/fr/manifestation/carmen-billet/idmanif/594946/idtier/18864121"
                "\"Charleston sur la Tour Eiffel\" au Trianon.\n"
                "   Prix: de 26,0 à 38,0 EUR\n\n"
                "Je veux ce format pour les événements, avec un petit extrait de la description de l'événement."
                "Il est très important de respecter le format demandé et de donner le top 3 des événements."
                
            )
        }
    ],
    stream=True,
)


for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

Voici les 3 événements qui correspondent à votre demande :

1. **Event: ANGE - L'ultime révérence du poète-chanteur**
   Date: 2025-01-15
   Lieu: L'Olympia
   Description: "Un rendez-vous imparable" - c'est ainsi qu'ANGE décrit son dernier concert, qui sera l'occasion de revivre les 55 ans d'une vie bien remplie.
   Prix: de 45.0 à 79.0 EUR

2. **Event: MAUVAIS OEIL - Premier escale**
   Date: 2025-01-25
   Lieu: La Maroquinerie
   Description: "Rendez-vous donc le 31 janvier 2025 à La Maroquinerie pour un concert unique qui marquera la « Première Escale » de notre voyage."
   Prix: de 29.0 à 35.0 EUR

3. **Event: MAU-D'AMOUR - Un nuage d'amour**
   Date: 2025-01-20
   Lieu: L'Olympia
   Description: "Dès les premières secondes, on ressent une émotion incroyable. Un spectacle puissant."
   Prix: de 17.0 à 27.5 EUR

In [13]:
import json
import ollama
import requests
from typing import Dict, List, Optional
import time
from datetime import datetime


class OllamaClient:
    def __init__(self, max_retries: int = 3):
        self.max_retries = max_retries

    def chat(self, model: str, messages: List[Dict[str, str]], stream: bool = False) -> Optional[Dict]:
        """Attempt to chat with Ollama with retry logic"""
        for attempt in range(self.max_retries):
            try:
                if stream:
                    response = []
                    stream_response = ollama.chat(
                        model=model,
                        messages=messages,
                        stream=True
                    )
                    for chunk in stream_response:
                        response.append(chunk.get('message', {}).get('content', ''))
                    return {'message': {'content': ''.join(response)}}
                else:
                    return ollama.chat(                    
                        model=model,
                        messages=messages
                    )
            except KeyboardInterrupt:
                print("\nOpération annulée par l'utilisateur.")
                return None
            except Exception as e:
                if attempt == self.max_retries - 1:
                    print(f"Échec de la communication avec Ollama après {self.max_retries} tentatives: {e}")
                    return None
                print(f"Tentative {attempt + 1} échouée, nouvelle tentative dans {2 ** attempt} secondes...")
                time.sleep(2 ** attempt)


class TicketmasterClient:
    def __init__(self, api_key: str, max_pages: int = 5):
        self.api_key = api_key
        self.max_pages = max_pages
        self.base_url = "https://app.ticketmaster.com/discovery/v2/events.json"

    def search_events(self, params: Dict) -> List[Dict]:
        """Search for events with pagination and error handling"""
        all_events = []
        current_page = 0

        while current_page < self.max_pages:
            try:
                search_params = {
                    **params,
                    "apikey": self.api_key,
                    "page": str(current_page),
                    "size": "20"
                }

                response = requests.get(
                    self.base_url,
                    params=search_params,
                    timeout=30
                )
                response.raise_for_status()

                data = response.json()
                events = data.get("_embedded", {}).get("events", [])

                if not events:
                    break

                for event in events:
                    event_info = self._parse_event(event)
                    if event_info:
                        all_events.append(event_info)

                current_page += 1

            except requests.RequestException as e:
                print(f"Erreur lors de la requête Ticketmaster: {e}")
                break
            except KeyboardInterrupt:
                print("\nRecherche interrompue par l'utilisateur.")
                break
            except Exception as e:
                print(f"Erreur inattendue: {e}")
                break

        return all_events

    def _parse_event(self, event: Dict) -> Optional[Dict]:
        """Parse a single event into a structured format"""
        try:
            event_info = {
                "name": event['name'],
                "date": event['dates']['start']['localDate'],
                "venue": event['_embedded']['venues'][0]['name'],
                "description": event.get('description', "Aucune description disponible"),
                "price": "Informations non disponibles",
                "url": event.get('url', "Lien non disponible")
            }

            if price_ranges := event.get("priceRanges"):
                min_price = price_ranges[0].get("min")
                max_price = price_ranges[0].get("max")
                currency = price_ranges[0].get("currency", "EUR")
                event_info["price"] = f"de {min_price} à {max_price} {currency}"

            return event_info
        except KeyError as e:
            print(f"Erreur lors du parsing d'un événement: {e}")
            return None


class EventSearch:
    def __init__(self, api_key: str):
        self.ollama_client = OllamaClient()
        self.ticketmaster_client = TicketmasterClient(api_key)
        self.system_message = (
            "Tu es un agent de voyage qui génère une liste d'événements "
            "pour aider à planifier un séjour. À partir d'une liste d'événements, "
            "tu dois donner le top 3 qui correspondent à la demande de l'utilisateur."
        )

    def generate_params(self, user_query: str) -> Optional[Dict]:
        """Generate search parameters from user query"""
        system_message = """Analyse cette requête utilisateur pour un événement et retourne uniquement un dictionnaire JSON avec les paramètres suivants, sans texte avant ou après:
        {
            "city": "ville mentionnée",
            "countryCode": "code pays (IT pour Italie, FR pour France, ES pour Espagne, etc.)",
            "locale": "*",
            "classificationName": "type d'événement (Music, Sports, Arts, Theatre, etc.)"
            "sort": "date,asc"
        }

        Si aucun type d'événement n'est spécifié, ne pas inclure classificationName.
        Si aucune ville n'est spécifiée, ne pas inclure city.
        """

        response = self.ollama_client.chat(
            model='llava:13b',
            messages=[
                {'role': 'system', 'content': system_message},
                {'role': 'user', 'content': user_query}
            ]
        )

        if not response:
            return None

        try:
            content = response['message']['content'].strip()
            start = content.find('{')
            end = content.rfind('}') + 1
            if start != -1 and end != -1:
                return json.loads(content[start:end])
            raise ValueError("No JSON found in response")
        except Exception as e:
            print(f"Erreur lors du parsing des paramètres: {e}")
            return None

    def generate_response(self, user_query: str, content: str, user_message_template: str) -> Optional[str]:
        """Generate a response based on events and user query"""
        formatted_message = user_message_template.format(content=content, user_query=user_query)

        response = self.ollama_client.chat(
            model='llama3.2-vision',
            messages=[
                {'role': 'system', 'content': self.system_message},
                {'role': 'user', 'content': formatted_message}
            ],
            stream=True
        )

        if response:
            return response['message']['content']
        return None

    def save_events(self, events: List[Dict], filename: str = "output.txt"):
        """Save events to a file"""
        try:
            with open(filename, "w", encoding="utf-8") as file:
                if events:
                    for event in events:
                        if event:  # Vérifier que l'événement est valide
                            file.write(f"Nom de l'événement: {event['name']}\n")
                            file.write(f"Date: {event['date']}\n")
                            file.write(f"Lieu: {event['venue']}\n")
                            file.write(f"Description: {event['description']}\n")
                            file.write(f"Prix: {event['price']}\n")
                            file.write(f"URL: {event['url']}\n")
                            file.write("-" * 50 + "\n")
                    print(f"{len([e for e in events if e])} événements trouvés et enregistrés dans '{filename}'")
                else:
                    file.write("Aucun événement trouvé pour ces critères de recherche.\n")
                    print("Aucun événement trouvé.")
        except IOError as e:
            print(f"Erreur lors de l'écriture du fichier: {e}")


def main():
    API_KEY = "97ViRGFgMAAjpbWjitt2p3JlLNfoMYQz"
    event_search = EventSearch(API_KEY)

    try:
        user_query = input("Veuillez entrer votre requête : ")
        params = event_search.generate_params(user_query)

        if not params:
            print("Impossible de générer les paramètres de recherche.")
            return

        print("Paramètres de recherche:", json.dumps(params, indent=2, ensure_ascii=False))
        events = event_search.ticketmaster_client.search_events(params)
        event_search.save_events(events)

        with open("output.txt", "r", encoding="utf-8") as file:
            content = file.read()

        custom_user_message = (
            "Voici les informations récupérées : {content}.\n\n"
            "Voici la demande de l'utilisateur : {user_query}\n\n"
            "Je veux que tu respectes le format suivant pour les événements :\n\n"
            "1. Event: Nom de l'événement\n"
            "   Date: Date de l'événement\n"
            "   Lieu: Lieu de l'événement\n"
            "   Description: Extrait de la description de l'événement\n"
            "   Lien Ticketmaster : Lien vers la billetterie\n"
            "   Prix: Prix de l'événement\n\n"
            "Il est essentiel de proposer le top 3 des événements qui correspondent à la demande "
            "de l'utilisateur, avec des dates différentes pour chaque événement et dans l'intervalle de dates demandé."
            "Propose des événements à des dates différentes sur la base des informations récupérées."
            "Je veux que tu fasses un proposition de planning avec les 3 events trouvés"
        )

        response = event_search.generate_response(user_query, content, custom_user_message)
        if response:
            print("\nRecommandations d'événements :")
            print(response)
        else:
            print("Impossible de générer des recommandations.")

    except KeyboardInterrupt:
        print("\nProgramme interrompu par l'utilisateur.")
    except Exception as e:
        print(f"Une erreur inattendue est survenue: {e}")


if __name__ == "__main__":
    main()

Paramètres de recherche: {
  "city": "Paris",
  "countryCode": "FR",
  "locale": "*",
  "classificationName": "Theatre",
  "sort": "date,asc"
}
100 événements trouvés et enregistrés dans 'output.txt'

Recommandations d'événements :
Après avoir parcouru les événements proposés par Ticketmaster, je vous propose trois options qui correspondent à votre demande :

1. **Event:** Le Trompe l'oeil, de 1520 à nos jours
   **Date:** Du 17 octobre 2024 au 02 mars 2025 (mais je peux vous proposer une date alternative pour avril ou mai)
   **Lieu:** Musée Marmottan Monet - Paris
   **Description:** L’exposition « Le trompe-l’œil, de 1520 à nos jours » retrace l’histoire et la diversité du trompe-l’œil, depuis son origine dans les années 1920 jusqu’à nos jours. L’exposition présente plus d’une centaine d’œuvres, entre autres.
   **Lien Ticketmaster :** https://www.ticketmaster.fr/fr/manifestation/musee-marmottan-monnet-paris
   **Prix:** de 12,5 à 16,5 EUR

2. **Event:** Nadia Léger – femme d’avant-

In [11]:
#je veux aller a paris entre Avril 2025 - Mai 2025  voir du theatre 

In [4]:
import requests
from scraper import EventbriteScraper
from ticket_master import request_ticket_master



# Deuxième scrapping
EventBrite = EventbriteScraper()
events_links = EventBrite.get_events_links("france", "paris", "soirée")
print(events_links)

# # Construire une chaîne de caractères avec les détails des événements
# events_text = ""
# for link in events_links:
#     print(link)
#     event_details = EventBrite.get_events_infos(link)
# 
#     events_text += f"Link: {link}\n"
#     for key, value in event_details.items():
#         events_text += f"{key.capitalize()}: {value}\n"
#     events_text += "\n" + "=" * 40 + "\n\n"  # Séparation entre les événements
# 
# # Écrire le texte dans un fichier .txt
# file_name = 'events.txt'
# with open(file_name, 'w', encoding='utf-8') as f:
#     f.write(events_text)
#     print(events_text)
# 
# EventBrite.quit_driver()

[]
